In [1]:
# ================================
# Smart Irrigation & Fertilizer Model
# ================================

import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score
import ipywidgets as widgets
from IPython.display import display, clear_output

# -------------------------------
# Load dataset
# -------------------------------
df = pd.read_excel("agro_smart_hilly_areas1.xlsx")
print("✅ Data Loaded. Shape:", df.shape)

# Fix column names if needed
if "crop_name" in df.columns and "crop_type" not in df.columns:
    df = df.rename(columns={"crop_name": "crop_type"})

display(df.head())

# -------------------------------
# Feature Engineering
# -------------------------------
features = [
    'soil_moisture_pct', 'temperature_C', 'humidity_pct',
    'rainfall_forecast_mm', 'soil_type', 'crop_type',
    'area_m2', 'flow_rate_lpm',
    'current_N_kg_ha', 'current_P_kg_ha', 'current_K_kg_ha'
]

features = [f for f in features if f in df.columns]
X = df[features]

# Check targets
has_labels = 'irrigation_needed' in df.columns and 'water_required_liters' in df.columns

# -------------------------------
# Pipelines
# -------------------------------
cat_cols = [c for c in ['soil_type', 'crop_type'] if c in X.columns]
num_cols = [c for c in X.columns if c not in cat_cols]

preprocess = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

clf_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("clf", RandomForestClassifier(n_estimators=200, random_state=42))
])

reg_pipeline = Pipeline([
    ("preprocess", preprocess),
    ("reg", RandomForestRegressor(n_estimators=200, random_state=42))
])

# -------------------------------
# Train Models (if labels exist)
# -------------------------------
if has_labels:
    y_clf = df['irrigation_needed']
    y_reg = df['water_required_liters']
    
    X_train, X_test, y_clf_train, y_clf_test, y_reg_train, y_reg_test = train_test_split(
        X, y_clf, y_reg, test_size=0.2, random_state=42
    )
    
    clf_pipeline.fit(X_train, y_clf_train)
    reg_pipeline.fit(X_train, y_reg_train)
    
    # Evaluate
    y_pred_clf = clf_pipeline.predict(X_test)
    y_pred_reg = reg_pipeline.predict(X_test)
    
    print("Classifier Accuracy:", accuracy_score(y_clf_test, y_pred_clf))
    print("Regressor MAE:", mean_absolute_error(y_reg_test, y_pred_reg))
    print("Regressor R²:", r2_score(y_reg_test, y_pred_reg))
    
    # Save models
    joblib.dump(clf_pipeline, "irrigation_classifier.joblib")
    joblib.dump(reg_pipeline, "water_regressor.joblib")
    print("✅ Models Trained & Saved!")
else:
    print("⚠️ No target labels found. Only heuristic fallback will be used.")

# -------------------------------
# Fertilizer Recommendation
# -------------------------------
fertilizer_reqs = {
    "rice": {"N": 120, "P": 60, "K": 40},
    "wheat": {"N": 100, "P": 50, "K": 50},
    "maize": {"N": 150, "P": 70, "K": 60},
    "barley": {"N": 90, "P": 40, "K": 30},
    "millet": {"N": 80, "P": 30, "K": 30},
    "potato": {"N": 180, "P": 90, "K": 100},
    "tea": {"N": 140, "P": 70, "K": 70},
}

def compute_fertilizer_recommendation(crop, area_m2, N, P, K):
    crop = str(crop).lower()
    if crop not in fertilizer_reqs:
        return {"N": 0, "P": 0, "K": 0}
    req = fertilizer_reqs[crop]
    area_ha = area_m2 / 10000
    addN = max(0, req["N"] * area_ha - N)
    addP = max(0, req["P"] * area_ha - P)
    addK = max(0, req["K"] * area_ha - K)
    return {"N": round(addN, 2), "P": round(addP, 2), "K": round(addK, 2)}

# -------------------------------
# Prediction Function
# -------------------------------
def predict_from_inputs(inputs):
    try:
        clf_model = joblib.load("irrigation_classifier.joblib")
        reg_model = joblib.load("water_regressor.joblib")
    except:
        clf_model, reg_model = None, None
    
    df_in = pd.DataFrame([inputs])
    
    if clf_model and reg_model:
        irr = int(clf_model.predict(df_in)[0])
        water_l = float(reg_model.predict(df_in)[0])
    else:
        irr = 1 if inputs["soil_moisture_pct"] < 30 and inputs["rainfall_forecast_mm"] < 20 else 0
        water_l = inputs["area_m2"] * (40 - inputs["soil_moisture_pct"]) if irr else 0
    
    pump_minutes = None
    if inputs.get("flow_rate_lpm", 0) > 0:
        pump_minutes = water_l / inputs["flow_rate_lpm"]
    
    fert = compute_fertilizer_recommendation(
        inputs["crop_type"], inputs["area_m2"],
        inputs["current_N_kg_ha"], inputs["current_P_kg_ha"], inputs["current_K_kg_ha"]
    )
    
    return {
        "irrigation_needed": irr,
        "water_required_liters": round(water_l, 2),
        "pump_on_minutes": round(pump_minutes, 2) if pump_minutes else None,
        "fertilizer_kg": fert
    }

# -------------------------------
# Input Form
# -------------------------------
soil_moisture = widgets.FloatText(description="Soil Moisture (%)", value=20)
temperature = widgets.FloatText(description="Temperature (°C)", value=30)
humidity = widgets.FloatText(description="Humidity (%)", value=50)
rainfall = widgets.FloatText(description="Rainfall Forecast (mm)", value=10)
soil_type = widgets.Dropdown(options=df['soil_type'].unique().tolist(), description="Soil Type")
crop_type = widgets.Dropdown(options=df['crop_type'].unique().tolist(), description="Crop Name")
area = widgets.FloatText(description="Land Area (m²)", value=1000)
flow_rate = widgets.FloatText(description="Flow Rate (L/min)", value=50)
N = widgets.FloatText(description="Current N (kg/ha)", value=50)
P = widgets.FloatText(description="Current P (kg/ha)", value=30)
K = widgets.FloatText(description="Current K (kg/ha)", value=20)

out = widgets.Output()

def on_submit(b):
    inputs = {
        "soil_moisture_pct": soil_moisture.value,
        "temperature_C": temperature.value,
        "humidity_pct": humidity.value,
        "rainfall_forecast_mm": rainfall.value,
        "soil_type": soil_type.value,
        "crop_type": crop_type.value,
        "area_m2": area.value,
        "flow_rate_lpm": flow_rate.value,
        "current_N_kg_ha": N.value,
        "current_P_kg_ha": P.value,
        "current_K_kg_ha": K.value,
    }
    result = predict_from_inputs(inputs)
    with out:
        clear_output()
        print("✅ Prediction Result:")
        print(result)

btn = widgets.Button(description="Run Prediction", button_style="success")
btn.on_click(on_submit)

form = widgets.VBox([
    soil_moisture, temperature, humidity, rainfall,
    soil_type, crop_type, area, flow_rate, N, P, K, btn, out
])

display(form)


✅ Data Loaded. Shape: (20000, 29)


,crop_type,crop_category,latitude,longitude,elevation_m,soil_type,slope_degree,aspect_degree,area_m2,crop_age_days,...,rainfall_next7_mm,weather_forecast,water_tank_level_l,rainwater_harvested_l,irrigation_required,valve_status,water_used_today_l,recommended_irrigation_liters,previous_yield_kg_per_ha,area_hectare
0,Blueberry,Fruit,27.143635,88.282500,1258.2,Silty Loam,31.2,36.8,10970,299,...,0.0,Sunny,12371.7,7977.6,Yes,Open,108.6,10036.7,4396.2,1.097
1,Coriander,Spice,27.287679,88.146128,1329.6,Clay,14.3,61.0,4330,276,...,0.0,Cloudy,4841.2,3523.4,Yes,Open,61.8,10291.5,1004.6,0.433
2,Raspberry,Fruit,27.232998,88.186660,1357.5,Peaty,23.7,316.6,5890,148,...,1.4,Light Rain,9325.9,2624.9,Yes,Open,9.3,10048.7,3776.7,0.589
3,Sugarcane,CashCrop,27.199665,88.265820,1350.2,Sandy Loam,18.1,178.1,11110,9,...,0.0,Cloudy,18698.3,13136.1,Yes,Open,155.6,2505.0,2826.3,1.111
4,Potato,Vegetable,27.089005,88.220522,1238.8,Sandy Loam,8.7,306.8,4240,194,...,0.0,Cloudy,9729.4,7411.0,No,Closed,0.0,0.0,3930.9,0.424


⚠️ No target labels found. Only heuristic fallback will be used.
